Log in to facebook?



In [7]:

var enterFacebook = () => {
    console.log('Facebook: Sign in required');
    var credentials = getCredentials('facebook.com');
    return client.click('input[name*="email"]')
    .keys(credentials.username)
    .pause(1000)
    .then(() => console.log('Facebook: Require password'))
    .click('input[name*="pass"]')
    .keys(credentials.password)
    .submitForm('[type="submit"]')
    .pause(2000)
    .isExisting('.cp-challenge-form')
    .then(is => {
        if(is) {
            throw new Error('captcha');
        }
    });
}

var loginFacebook = () => {
    return client
    .getUrl()
    .then(url => {
        var loggedIn = url.indexOf('facebook') > -1 && url.indexOf('login') == -1;
        return loggedIn
            ? client
                .isVisible('input[name*="email"]')
                .then(is => is ? enterFacebook() : client)
            : client.url('https://www.facebook.com/')
                .isVisible('input[name*="email"]')
                .then(is => is ? enterFacebook() : client)
    })
}
module.exports = loginFacebook;
loginFacebook;



[Function: loginFacebook]

List Facebook threads?



In [3]:
var scrollFacebookThreads = (threads) => {
    return client
    .execute(() => {
        document.getElementsByClassName(
            'scrollable')[0]
            .scrollTop += 10000;
    })
    .pause(2000)
    .then(() => getFacebookThreads(threads))
};

var getFacebookThreads = (threads) => {
    return client    
    .execute(() => {
        var iterator = document.evaluate(
            '//*[contains(@data-href, "messages")]/@data-href',
            document, null,
            XPathResult.UNORDERED_NODE_ITERATOR_TYPE, null);
        var co = []; var m;
        while ((m = iterator.iterateNext()) && co.push(m));
        return co;
    })
    .then(els => {
        var newThreads = els.value.filter(e => threads.indexOf(e) === -1);
        threads = threads.concat(newThreads);
        return newThreads.length > 0
            ? scrollFacebookThreads(threads)
            : Promise.resolve(threads);
    })
};

var listFacebookThreads = (threads) => {
    throw new Error('dont call this');
    return client.getUrl()
    .then(url => url.indexOf('/messages/t') == -1
        ? client
              .click('[data-tooltip-content="Messages"]')
              .click('a[href*="/messages/t"]')
        : Promise.resolve([]))
    .pause(2000)
    .then(() => getFacebookThreads(threads || []))
    .then(r => threads = r)
    .then(url => url.indexOf('/messages/archived') == -1
        ? client.url('https://www.facebook.com/messages/archived')
              .pause(1000)
              .click('.scrollable a[href="#"]')
        : Promise.resolve([]))
    .pause(2000)
    .then(() => getFacebookThreads(threads || []))
};
module.exports = listFacebookThreads;
listFacebookThreads;



[Function: listFacebookThreads]

Scrape facebook profile?
    
    

In [2]:
var importer = require('../Core');

// TODO: pass profile path in as parameter becomes readFacebookProfileInfo
var readFacebookProfileInfo = () => {
    var profile = {};
    // TODO: check for profile path
    return client.execute(() => {
        var people = document.evaluate(
            '//*[contains(@class, "scrollable")]//h4[contains(., "Profile")]/parent::*//a',
            document, null,
            XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
        if(!people) {
            return '';
        }
        people.setAttribute('target', '_self');
        return people;
    })
    .then(r => r.value != '' 
        ? client.elementIdClick(r.value.ELEMENT)
            .pause(2000)
            .then(() => importer.runAllPromises([
                client.getText('[role="main"] h1'),
                client.isExisting('#intro_container_id li').then(is => is 
                    ? client.getText('#intro_container_id li') 
                    : Promise.resolve('')),
                client.getUrl(),
            ]))
            .then(r => (profile = ({
                name: r[0],
                description: r[1],
                url: r[2],
            })))
        : client)
    .then(() => profile)
    .catch(e => console.log(e))
};
module.exports = readFacebookProfileInfo;
readFacebookProfileInfo;



[Function: readFacebookProfileInfo]

Get messages from facebook?



In [1]:
var importer = require('../Core');
var moment = require('moment');
var chrono = require('chrono-node');

var readFacebookProfileInfo;
var switchToParticipantThread = (i) => {
    return client.execute((i) => {
        var people = document.evaluate(
            '//*[contains(@class, "scrollable")]//h4[contains(., "People")]/parent::*//li[' + i + ']',
            document, null,
            XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue
        return people;
    }, i)
    .then(r => client.elementIdClick(r.value.ELEMENT))
    .pause(1000)
    .then(() => readFacebookProfileInfo())
    .catch(e => console.log(e))
};

var getThreadParticipants = (thread) => {
    return client.then(() => importer.interpretAll(['scrape facebook profile']))
    .then(r => r[0].runFresherInNewContext({client}))
    .then(r => readFacebookProfileInfo = r)
    .execute(() => {
        var people = document.evaluate(
            'count(//*[contains(@class, "scrollable")]//h4[contains(., "People")]/parent::*//li)',
            document, null,
            XPathResult.NUMBER_TYPE, null).numberValue;
        return people;
    })
    .then(r => {
        var result = [];
        for(var i = 1; i < Math.min(3, r.value); i++) {
            result[result.length] = (i =>
                resolve => {
                var profile = {};
                switchToParticipantThread(i)
                .then(r => profile = r)
                .url(thread)
                .pause(3000)
                .catch(e => console.log(e))
                .then(() => resolve(profile))
            })(i);
        }
        if(r.value === 0) {
            result[result.length] = (resolve => {
                var profile = {};
                readFacebookProfileInfo()
                .then(r => profile = r)
                .url(thread)
                .pause(3000)
                .then(() => resolve(profile))
                .catch(e => console.log(e))
            })
        }
        console.log(result);
        return importer.runAllPromises(result).catch(e => console.log(e));
    })
    .catch(e => console.log(e))
};

var scrollFacebookMessages = (messages, i) => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    // scroll to bottom of messages
    return client
    .execute(() => {
        document.getElementsByClassName('scrollable')[2]
            .scrollTop -= 10000;
    })
    .pause(2000)
    .then(() => i > 3 ? messages : readFacebookMessages(messages, i + 1))
    .catch(e => console.log(e))
};

var readFacebookMessages = (messages, i = 0) => {
    return client
    .execute(() => {
        var result = [];
        var groups = document.getElementsByClassName('scrollable')[2];
        var times = groups.getElementsByTagName('h4');
        for(var t = 0; t < times.length; t++) {
            var bubble = times[t];
            while((bubble = bubble.nextSibling) && bubble.nodeName == 'DIV') {
                var from = bubble.getElementsByTagName('h5')[0];
                result[result.length] = {
                    from: typeof from !== 'undefined' ? from.textContent : '',
                    time: times[t].innerText,
                    message: bubble.innerText
                };
            }
        }
        return result;
    })
    .then(r => {
        var newMessages = r.value
            .filter(e => messages
                .filter(m => m.message == e.message).length === 0);
        messages = newMessages.concat(messages);
        return newMessages.length > 0
                ? scrollFacebookMessages(messages, i)
                : Promise.resolve(messages);
    })
    .catch(e => console.log(e))
};

var readFacebookThread = (thread, messages) => {
    var participants;
    return client.getUrl().then(url => url.indexOf(thread) == -1
        ? client.url(thread)
        : client)
    // get participants from topcard
    .pause(1000)
    .then(() => getThreadParticipants(thread))
    .then(p => (participants = p))
    
    // TODO: save to contacts
    .then(() => readFacebookMessages(messages || []))
    .then(messages => ({
        thread: thread,
        participants: participants || [],
        messages: messages.map(m => {
            var newTime = chrono.parseDate(m.time);
            newTime.setHours(newTime.getHours()-(new Date()).getTimezoneOffset()/60)
            return Object.assign(m, {time: newTime})
        })
    }))
    .catch(e => console.log(e))
};
module.exports = readFacebookThread;
readFacebookThread;



[Function: readFacebookThread]

In [ ]:
var importer = require('../Core');
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var readThread = (thread) => {
    var threadId = thread.replace(/^\/|\/$/ig, '').split('/').pop();
    var file = glob.sync('**/' + threadId + '-*.json', {cwd: project})[0];
    var messages;
    try { messages = require(file) }
    catch (e) { messages = [] }
    return readFacebookThread(thread, messages)
    .then(t => {
        var filename = project + '/' 
            + t.thread.replace(/^\/|\/$/ig, '').split('/')
            .pop().replace(/[^a-z0-9]/ig, '_')
            + '-' + t.participants
            .map(p => p.name).join('')
            .replace(/[^a-z0-9]/ig, '_') + '.json';
        fs.writeFileSync(filename, JSON.stringify(t, null, 4));
        return t;
    }).catch(e => console.log(e));
};

var listThreads = () => {
    return runSeleniumCell('list facebook threads')
    .then(listFacebookThreads => {
        return listFacebookThreads(threads);
    })
    // list all message threads
    .then(r => fs.writeFileSync(
        project + '/facebook-threads.json',
        JSON.stringify(r, null, 4)))
};

$$.async()
var threads, readFacebookThread;
var runSeleniumCell;

importer.interpretAll(['selenium cell'])
.then(r => {
    runSeleniumCell = r[0].runInNewContext();
    return runSeleniumCell('log in facebook');
})
.then(loginFacebook => loginFacebook())
.then(() => {
    try { threads = JSON.parse(fs.readFileSync(project + '/facebook-threads.json')); }
    catch (e) { threads = []; }
    return runSeleniumCell('messages from facebook');
})
//.then(() => listThreads())
.then(func => {
    readFacebookThread = func;
    return importer.runAllPromises(threads.slice(200, 600)
        .map(t => ((resolve) => readThread(t).then((r) => resolve(r)))));
})
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e))



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
[ [Function] ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
[ [Function] ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
[ [Function] ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
[ [Function] ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
[ [Function] ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipyn